In [1]:
directory = 'C:/Users/eekcl/clomoblade Folder/pytest/2008'  # 사진 파일들이 있는 디렉토리 경로로 변경


In [4]:
import os
from PIL import Image
from PIL.ExifTags import TAGS
from datetime import datetime

def get_exif_date(image_path):
    try:
        image = Image.open(image_path)
        exif_data = image._getexif()
        if exif_data:
            for tag, value in exif_data.items():
                decoded = TAGS.get(tag, tag)
                if decoded == 'DateTimeOriginal':
                    try:
                        return datetime.strptime(value, '%Y:%m:%d %H:%M:%S')
                    except ValueError:
                        pass
    except Exception as e:
        print(f"Error reading EXIF data from {image_path}: {e}")
    return None

def get_creation_date(image_path):
    t = os.path.getctime(image_path)
    return datetime.fromtimestamp(t)

def get_modification_date(image_path):
    t = os.path.getmtime(image_path)
    return datetime.fromtimestamp(t)

def get_earliest_date(dates):
    return min(date for date in dates if date is not None)

def generate_new_filename(directory, date_str, count, extension):
    new_filename = f"{date_str}_{count:03d}{extension}"
    new_file_path = os.path.join(directory, new_filename)
    while os.path.exists(new_file_path):
        count += 1
        new_filename = f"{date_str}_{count:03d}{extension}"
        new_file_path = os.path.join(directory, new_filename)
    return new_filename, count

def rename_files(directory):
    files = []
    
    # 사진 파일의 날짜 정보 가져오기
    for filename in os.listdir(directory):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif', 'mp4')):
            file_path = os.path.join(directory, filename)
            exif_date = get_exif_date(file_path)
            creation_date = get_creation_date(file_path)
            modification_date = get_modification_date(file_path)
            earliest_date = get_earliest_date([exif_date, creation_date, modification_date])
            if earliest_date:
                try:
                    file_size = os.path.getsize(file_path)
                    files.append((file_path, earliest_date, file_size))
                except Exception as e:
                    print(f'Error processing {filename}: {e}')

    # 날짜별로 파일들을 분류하고 정렬
    files_by_date = {}
    for file_path, earliest_date, file_size in files:
        date_str = earliest_date.strftime('%Y%m%d')
        if date_str not in files_by_date:
            files_by_date[date_str] = []
        files_by_date[date_str].append((file_path, earliest_date, file_size))
    
    # 날짜별로 파일명을 변경
    for date_str, files in files_by_date.items():
        files.sort(key=lambda x: (x[1], x[2]))  # 날짜와 파일 크기로 정렬
        count = 1
        previous_time = None
        for file_path, earliest_date, file_size in files:
            current_time = earliest_date.strftime('%H:%M:%S')
            if current_time == previous_time:
                count += 1
            else:
                count = 1
                previous_time = current_time
            extension = os.path.splitext(file_path)[1]
            new_filename, count = generate_new_filename(directory, date_str, count, extension)
            new_file_path = os.path.join(directory, new_filename)
            os.rename(file_path, new_file_path)
            print(f'Renamed: {os.path.basename(file_path)} to {new_filename}')

#directory = 'your_directory_path_here'  # 사진 파일들이 있는 디렉토리 경로로 변경
rename_files(directory)


Error reading EXIF data from C:/Users/eekcl/clomoblade Folder/pytest/2008\20080403_002.bmp: 'BmpImageFile' object has no attribute '_getexif'
Renamed: 20080103_003.jpg to 20080103_001.jpg
Renamed: 20080105_003.jpg to 20080105_002.jpg
Renamed: 20080105_004.jpg to 20080105_003.jpg
Renamed: 20080105_006.jpg to 20080105_004.jpg
Renamed: 20080105_001.jpg to 20080105_005.jpg
Renamed: 20080108_001.jpg to 20080108_002.jpg
Renamed: 20080108_003.jpg to 20080108_001.jpg
Renamed: 20080110_002.jpg to 20080110_003.jpg
Renamed: 20080110_006.jpg to 20080110_002.jpg
Renamed: 20080110_008.jpg to 20080110_005.jpg
Renamed: 20080110_001.jpg to 20080110_006.jpg
Renamed: 20080110_010.jpg to 20080110_001.jpg
Renamed: 20080110_004.jpg to 20080110_007.jpg
Renamed: 20080111_001.jpg to 20080111_003.jpg
Renamed: 20080111_011.jpg to 20080111_001.jpg
Renamed: 20080111_012.jpg to 20080111_005.jpg
Renamed: 20080111_014.jpg to 20080111_008.jpg
Renamed: 20080111_002.jpg to 20080111_010.jpg
Renamed: 20080111_004.jpg to 2

In [26]:
import os
from datetime import datetime
from hachoir.metadata import extractMetadata
from hachoir.parser import createParser

def get_datetime_original(file_path):
    try:
        parser = createParser(file_path)
        if not parser:
            print(f"Unable to parse file {file_path}")
            return None
        with parser:
            metadata = extractMetadata(parser)
            if metadata and metadata.has("creation_date"):
                return metadata.get("creation_date")
    except Exception as e:
        print(f"Error reading datetimeoriginal from {file_path}: {e}")
    return None

def generate_new_filename(directory, date_str, count, extension):
    new_filename = f"{date_str}_{count:03d}{extension}"
    new_file_path = os.path.join(directory, new_filename)
    while os.path.exists(new_file_path):
        count += 1
        new_filename = f"{date_str}_{count:03d}{extension}"
        new_file_path = os.path.join(directory, new_filename)
    return new_filename, count

def rename_files(directory):
    files = []
    
    # MP4 파일의 datetimeoriginal 정보 가져오기
    for filename in os.listdir(directory):
        if filename.lower().endswith('.mp4'):
            file_path = os.path.join(directory, filename)
            datetime_original = get_datetime_original(file_path)
            if datetime_original:
                try:
                    file_size = os.path.getsize(file_path)
                    files.append((file_path, datetime_original, file_size))
                except Exception as e:
                    print(f'Error processing {filename}: {e}')

    # 날짜별로 파일들을 분류하고 정렬
    files_by_date = {}
    for file_path, datetime_original, file_size in files:
        date_str = datetime_original.strftime('%Y%m%d')
        if date_str not in files_by_date:
            files_by_date[date_str] = []
        files_by_date[date_str].append((file_path, datetime_original, file_size))
    
    # 날짜별로 파일명을 변경
    for date_str, files in files_by_date.items():
        files.sort(key=lambda x: (x[1], x[2]))  # 날짜와 파일 크기로 정렬
        count = 1
        previous_time = None
        for file_path, datetime_original, file_size in files:
            current_time = datetime_original.strftime('%H:%M:%S')
            if current_time == previous_time:
                count += 1
            else:
                count = 1
                previous_time = current_time
            extension = os.path.splitext(file_path)[1]
            new_filename, count = generate_new_filename(directory, date_str, count, extension)
            new_file_path = os.path.join(directory, new_filename)
            try:
                os.rename(file_path, new_file_path)
                print(f'Renamed: {os.path.basename(file_path)} to {new_filename}')
            except Exception as e:
                print(f'Error renaming {file_path} to {new_filename}: {e}')

#directory = 'your_directory_path_here'  # MP4 파일들이 있는 디렉토리 경로로 변경
rename_files(directory)


Renamed: 19040101_001.mp4 to 19040101_004.mp4
Renamed: 19040101_002.mp4 to 19040101_005.mp4
Renamed: 19040101_003.mp4 to 19040101_006.mp4
Renamed: 20220608_002.mp4 to 20220608_001.mp4
Renamed: 20220702_002.mp4 to 20220702_001.mp4
Renamed: 20220703_002.mp4 to 20220703_001.mp4
Renamed: 20220709_003.mp4 to 20220709_002.mp4
Renamed: 20220709_001.mp4 to 20220709_003.mp4
Renamed: 20220723_002.mp4 to 20220723_001.mp4
Renamed: 20220810_003.mp4 to 20220810_002.mp4
Renamed: 20220810_001.mp4 to 20220810_003.mp4
Renamed: 20220827_002.mp4 to 20220827_001.mp4
Renamed: 20220903_002.mp4 to 20220903_001.mp4
